In [1]:
import scanpy as sc
import ccAFv2

D:\Python_environments\C_drive\Anaconda3\envs\ccaf_v2_gpu\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [2]:

# Load the data from your CSV file
import scanpy as sc

# Path to the directory containing the 10x files
adata = sc.read_10x_mtx("D:/Halima's Data/Thesis_2/GSE_temporary/", var_names='gene_symbols', cache=True)

adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pp.filter_genes(adata, min_cells=1)  # Keeps only genes that are expressed in at least 1 cell

# Normalization and log transformation
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)



In [20]:
adata

AnnData object with n_obs × n_vars = 1152 × 32970
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_ids', 'feature_types', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells'
    uns: 'log1p'

In [3]:
# Predict cell cycle phases
predicted_labels, probabilities = ccAFv2.predict_labels(adata, species='human', gene_id='symbol')

# Add predictions to the AnnData object
adata.obs['ccAFv2'] = predicted_labels


Running ccAFv2:
    Preparing data for classification...
    Marker genes present in this dataset: 833
    Missing marker genes in this dataset: 28
  Predicting cell cycle state probabilities...
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
  Choosing cell cycle state...
Done.


In [4]:
print(adata.obs[['ccAFv2']].head())


        ccAFv2
A10_355     G1
A10_356     G1
A10_357     G1
A11_355     G1
A11_356     G1


In [7]:
# Save predictions to a CSV file
adata.obs[['ccAFv2']].to_csv("D:/Halima's Data/Thesis_2/RCode/Cell_Cycle_prediction_with_scATAC_Seq/paper1/Training_data/Benchmark_data/predicted_ccAFV2_64016_cell_cycle_labels.csv")
